In [7]:
import pandas as pd
from pprint import pprint
from datasets import load_dataset

dataset = load_dataset("weerayut/iq-data")
dataset

DatasetDict({
    validation: Dataset({
        features: ['conver_id', 'pattern', 'sampling_stergy', 'turn_ids', 'questions', 'answers', 'facts', 'retrieved_facts', 'citations'],
        num_rows: 100
    })
})

## To dataset format

In [8]:
from pathlib import Path
from src.utils import FormatPrompt, DialogueManager

system_prompt = Path(f"src/prompt/context_with_citations.md").read_text().strip()
FP = FormatPrompt("contextcitations")
## 1) INPUT:    user, meta_user = FP.format_input(q, facts=facts)
## 2) Extract:  FP.extract_output(out)
## 3) Format:   assistant, meta_assistant = FP.format_output(LLMs['answer'], LLMs['citations'])
DM = DialogueManager(system_prompt=system_prompt, log_path=".cache/chat.log")    
## 2) GET:      turn = DM.data() + [user]  => 
## 3) ADD Q/A:  DM.add([meta_user, meta_assistant]) 
## DM.reset() and DM.save()

In [9]:
data = dataset["validation"].to_pandas()

In [ ]:
last_turn_dataset = []
for (conver_id, pattern, sampling_id), conversation in data.groupby(["conver_id", "pattern", "sampling_stergy"]):
    conv_id = f"{conver_id}_{pattern}_{sampling_id}"

    converstaions = []
    conv = eval(conversation.to_json(orient="records", force_ascii=False,))
    
    DM = DialogueManager(system_prompt=system_prompt, log_path=".cache/chat.log")    
    for i, items in enumerate(conv):
        Q = items['questions']
        A = items['answers']
        F = items['retrieved_facts']
        C = items['citations']

        ## INPUT ##
        user, meta_user = FP.format_input(Q, facts=F) 
        assistant, meta_assistant = FP.format_output(A, C)
        
        ## Keep Conv
        turn = DM.data() + [user, assistant]
        converstaions.append(turn)
        
        ## Add history
        meta_user = {k:v for k,v in meta_user.items() if k in ["role", "content"]}
        meta_assistant = {k:v for k,v in meta_assistant.items() if k in ["role", "content"]}
        DM.add([meta_user, meta_assistant]) 
        DM.save()
    
    question, answer = converstaions[-1][:-1], converstaions[-1][-1]
    last_turn_dataset.append({"id": f"{conv_id}_turn_{i}", "question": question, "answer": answer,})

In [11]:
ds = pd.DataFrame(last_turn_dataset)
ds

,id,question,answer
0,10_CCCC_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
1,18_CPDR_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
2,30_PDDD_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
3,53_DDPD_1_turn_2,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
4,84_PDDP_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
5,95_CCCC_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
6,98_PDDD_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
7,163_DCPR_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
8,195_DDPR_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."
9,198_DPPD_1_turn_4,"[{'role': 'system', 'content': 'Answer only in...","{'role': 'assistant', 'content': '{""answer"": ""..."


## Check data

In [12]:
index = 3

In [13]:
pprint(last_turn_dataset[index]['question'], sort_dicts=False, width=300)

[{'role': 'system', 'content': 'Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.'},
 {'role': 'user', 'content': 'ความเครียดส่งผลต่อระบบภูมิคุ้มกันของร่างกายอย่างไร'},
 {'role': 'assistant', 'content': 'ความเครียดที่สะสมหรือเรื้อรังส่งผลให้ระบบภูมิคุ้มกันอ่อนแอลง เนื่องจากฮอร์โมนความเครียดที่หลั่งเพิ่มขึ้นจะยับยั้งการทำงานของระบบภูมิคุ้มกัน ทำให้ร่างกายป่วยง่ายขึ้น ติดเชื้อได้ง่ายและฟื้นตัวช้ากว่าปกติ'},
 {'role': 'user', 'content': 'ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนใดมากที่สุด'},
 {'role': 'assistant', 'content': 'ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนไฮโปธาลามัสและสมองส่วนที่เกี่ยวข้องกับความจำและสมาธิ โดยทำให้เกิดความผิดปกติในการหลั่งฮอร์โมนเจริญพันธุ์ ไข่ไม่ตก มีสมาธิสั้น และความจำแย่ลง เนื่องจากความเครียดมีผลโดยตรงต่อการทำงานของระบบประสาทส่วนกลาง'},
 {'role': 'user',
  'content': 'คำถาม:\n'
             'ความเครียดเรื้อรังมีผลต่อฮอร์โมนคอร์ติซอลอย่างไร และส่งผลต่อระ

In [14]:
pprint(last_turn_dataset[index]['answer'], sort_dicts=False, width=300)

{'role': 'assistant',
 'content': '{"answer": "ความเครียดเรื้อรังทำให้ร่างกายหลั่งฮอร์โมนคอร์ติซอลมากเกินไปจากต่อมหมวกไต โดยเริ่มต้นจากการที่ความเครียดกระตุ้นสมองส่วนไฮโปธาลามัส ซึ่งควบคุมการหลั่งฮอร์โมนในระบบประสาทและต่อมไร้ท่อ '
            'ส่งผลให้ระบบประสาทส่วนกลางสั่งการให้ต่อมใต้สมองและต่อมหมวกไตหลั่งฮอร์โมนคอร์ติซอลอย่างต่อเนื่อง ซึ่งฮอร์โมนนี้เมื่ออยู่ในระดับสูงเป็นเวลานานจะยับยั้งการทำงานของเซลล์ภูมิคุ้มกัน ทำให้ระบบภูมิคุ้มกันอ่อนแอลง ร่างกายติดเชื้อง่ายขึ้น และฟื้นตัวจากโรคได้ช้าลง", "citations": [1, 2, 3, 4, '
            '5]}'}


## Check turns

In [15]:
pprint(converstaions[1], sort_dicts=False, width=300)

[{'role': 'system', 'content': 'Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.'},
 {'role': 'user', 'content': 'อาการใดที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของโรคไข้เลือดออก?'},
 {'role': 'assistant',
  'content': 'อาการที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของโรคไข้เลือดออกคือ การมีอาการเป็นลมหรือหมดสติจนไม่รู้สึกตัว ซึ่งเป็นสัญญาณบ่งชี้ถึงภาวะวิกฤตที่ต้องได้รับการรักษาพยาบาลอย่างเร่งด่วน โดยระยะวิกฤตมักเกิดขึ้นหลังระยะไข้ประมาณ 48–72 ชั่วโมง '
             'และอาจมีความผิดปกติของการไหลเวียนเลือดหรือภาวะช็อกได้ หากไม่ได้รับการดูแลทันทีอาจนำไปสู่ภาวะแทรกซ้อนรุนแรง'},
 {'role': 'user',
  'content': 'คำถาม:\n'
             'อะไรคือปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกมีโอกาสเข้าสู่ระยะวิกฤตได้เร็วขึ้น?\n'
             'Facts:\n'
             '[1] เด็กอายุต่ำกว่า 1 ปีและผู้สูงอายุมีความเสี่ยงที่จะเกิดภาวะรุนแรงหรือภาวะแทรกซ้อนจากไข้เลือดออกได้สูงกว่า\n'
             '[2] ปัจจัยเสี่ยงต่อเล

## Apply with chat template

In [16]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True,)

# Build prompt
text = tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True,)
print(text)

<|im_start|>system
Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.<|im_end|>
<|im_start|>user
อาการใดที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของโรคไข้เลือดออก?<|im_end|>
<|im_start|>assistant
อาการที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของโรคไข้เลือดออกคือ การมีอาการเป็นลมหรือหมดสติจนไม่รู้สึกตัว ซึ่งเป็นสัญญาณบ่งชี้ถึงภาวะวิกฤตที่ต้องได้รับการรักษาพยาบาลอย่างเร่งด่วน โดยระยะวิกฤตมักเกิดขึ้นหลังระยะไข้ประมาณ 48–72 ชั่วโมง และอาจมีความผิดปกติของการไหลเวียนเลือดหรือภาวะช็อกได้ หากไม่ได้รับการดูแลทันทีอาจนำไปสู่ภาวะแทรกซ้อนรุนแรง<|im_end|>
<|im_start|>user
อะไรคือปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกมีโอกาสเข้าสู่ระยะวิกฤตได้เร็วขึ้น?<|im_end|>
<|im_start|>assistant
ปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกมีโอกาสเข้าสู่ระยะวิกฤตได้เร็วขึ้น ได้แก่ อายุของผู้ป่วย โดยเฉพาะเด็กอายุต่ำกว่า 1 ปีและผู้สูงอายุ ซึ่งมีความเสี่ยงสูงต่อการเกิดภาวะรุนแรงหรือภาวะแทรกซ้อนจากโรค ทำให้อาจเข้าสู่ระยะวิกฤตได้เร็วก

In [17]:
pprint(answer)

{'content': '{"answer": "ในเด็กเล็กที่ติดเชื้อไข้เลือดออก ควรสังเกตอาการเช่น '
            'ไข้สูงร่วมกับปวดศีรษะ ปวดเบ้าตา คลื่นไส้อาเจียน ผื่น '
            'หรือมีภาวะเลือดไปเลี้ยงไม่ดี เช่น ปลายมือปลายเท้าเย็น '
            'การหายใจเร็วหรือเหนื่อย อุณหภูมิร่างกายต่ำผิดปกติ '
            'หรืออาการเป็นพิษ เช่น ซึม หมดสติ '
            'รวมถึงผื่นจุดเลือดออกหรือผื่นเลือดคั่ง '
            'ซึ่งเป็นสัญญาณเตือนที่อาจนำไปสู่ระยะวิกฤตได้หากไม่ได้รับการดูแลทันที", '
            '"citations": [1, 5, 6]}',
 'role': 'assistant'}


## OPENAI

In [18]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4o",
    messages=question
)

print(response.choices[0].message.content)

{"answer":"อาการที่ควรสังเกตในเด็กเล็กที่ติดเชื้อไข้เลือดออกเพื่อป้องกันการเข้าสู่ระยะวิกฤตอย่างทันท่วงทีได้แก่ ไข้สูงเรื้อรังที่ไม่ลดลง อาการซึม เลือดไปเลี้ยงไม่ดี ซึ่งแสดงออกโดย capillary refill time ที่มากกว่า 2 วินาที มือเท้าเย็น การเปลี่ยนแปลงสีผิว หรือการร้องไห้ที่มีคุณภาพผิดปกติ อาการเหล่านี้ชี้ให้เห็นถึงภาวะช็อกหรือการไหลเวียนเลือดที่ผิดปกติอันเป็นสัญญาณของระยะวิกฤติในไข้เลือดออก ควรพาเด็กไปพบแพทย์ทันทีเมื่อพบอาการเหล่านี้ [1][2][7]","citations":["1","2","7"]}
